In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

#随机初始化函数
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


mb_size = 32
Z_dim = 16
c_dim = 10

X = tf.placeholder(tf.float32, shape=[None, 784])

#判别器中间参数
D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

DC_D_W1 = tf.Variable(xavier_init([5, 5, 1, 1]))
DC_D_b1 = tf.Variable(tf.zeros(shape=[1]))

DC_D_W2 = tf.Variable(xavier_init([3, 3, 1, 1]))
DC_D_b2 = tf.Variable(tf.zeros(shape=[1]))

DC_D_W3 = tf.Variable(xavier_init([7 * 7 * 1, 128]))
DC_D_b3 = tf.Variable(tf.zeros(shape=[128]))

DC_D_W4 = tf.Variable(xavier_init([128, 1]))
DC_D_b4 = tf.Variable(tf.zeros(shape=[1]))

theta_DC_D = [DC_D_W1, DC_D_b1, DC_D_W2, DC_D_b2, DC_D_W3, DC_D_b3, DC_D_W4, DC_D_b4]

#噪声
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])
#隐含因子latent code
c = tf.placeholder(tf.float32, shape=[None, c_dim])

#生成器中间参数
G_W1 = tf.Variable(xavier_init([Z_dim+c_dim, 256]))
G_b1 = tf.Variable(tf.zeros(shape=[256]))

G_W2 = tf.Variable(xavier_init([256, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

#Q参数
Q_W1 = tf.Variable(xavier_init([784, 128]))
Q_b1 = tf.Variable(tf.zeros(shape=[128]))

Q_W2 = tf.Variable(xavier_init([128, c_dim]))
Q_b2 = tf.Variable(tf.zeros(shape=[c_dim]))

theta_Q = [Q_W1, Q_W2, Q_b1, Q_b2]


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def sample_c(m):
    return np.random.multinomial(1, c_dim*[1/c_dim], size=m)


def generator(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob


def dc_discriminator(x):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    conv1 = tf.nn.relu(tf.nn.conv2d(x, DC_D_W1, strides=[1, 2, 2, 1], padding='SAME') + DC_D_b1)
    conv2 = tf.nn.relu(tf.nn.conv2d(conv1, DC_D_W2, strides=[1, 2, 2, 1], padding='SAME') + DC_D_b2)
    conv2 = tf.reshape(conv2, shape=[-1, 7 * 7 * 1])
    h = tf.nn.relu(tf.matmul(conv2, DC_D_W3) + DC_D_b3)
    logit = tf.matmul(h, DC_D_W4) + DC_D_b4
    prob = tf.nn.sigmoid(logit)

    return prob

def Q(x):
    Q_h1 = tf.nn.relu(tf.matmul(x, Q_W1) + Q_b1)
    Q_prob = tf.nn.softmax(tf.matmul(Q_h1, Q_W2) + Q_b2)

    return Q_prob


def plot(samples):
    fig = plt.figure(figsize=(10, c_dim))
    gs = gridspec.GridSpec(c_dim, 10)
    gs.update(wspace=0.05, hspace=0.05)
    
    for a in range(c_dim):
        for i, sample in enumerate(samples[a]):
            ax = plt.subplot(gs[i+a*c_dim])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    
    return fig

G_sample = generator(Z, c)
D_real = discriminator(X)
D_fake = discriminator(G_sample)
Q_c_given_x = Q(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real + 1e-8) + tf.log(1 - D_fake + 1e-8))
G_loss = -tf.reduce_mean(tf.log(D_fake + 1e-8))

# D_real, D_logit_real = dc_discriminator(X)
# D_fake, D_logit_fake = dc_discriminator(G_sample)
# D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
# D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
# D_loss = D_loss_real + D_loss_fake
# G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

cross_ent = tf.reduce_mean(-tf.reduce_sum(tf.log(Q_c_given_x + 1e-8) * c, 1))
Q_loss = cross_ent

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)
Q_solver = tf.train.AdamOptimizer().minimize(Q_loss, var_list=theta_Q + theta_G)

mnist = input_data.read_data_sets('./MNIST', one_hot=True)

sess = tf.Session()
summary_writer = tf.summary.FileWriter("infogan_logs/logs10/", sess.graph)
sess.run(tf.global_variables_initializer())

if not os.path.exists('dcout10/'):
    os.makedirs('dcout10/')

i = 0

for it in range(1000000):
    #每1000步输出一张图片
    if it % 1000 == 0:
        Z_noise = sample_Z(10, Z_dim)

        idx = np.random.randint(0, c_dim) #0-11的整数
#         c_noise = np.zeros([10, 12])
#         c_noise[range(10), 0] = 1   #令c_noise某一列为1
        
#         samples = sess.run(G_sample,
#                            feed_dict={Z: Z_noise, c: c_noise})
        sample_list = []
        for n in range(c_dim):
            c_noise = np.zeros([10,c_dim])
            c_noise[range(10),n] = 1
            samples = sess.run(G_sample,
                              feed_dict={Z: Z_noise, c: c_noise})
            sample_list.append(samples)

        #Tensorboard可视化
        with tf.name_scope('fake_image'):
            image_shaped = tf.reshape(samples[0],[-1,28,28,1])
            tf.summary.image('fake_image',image_shaped,10)
        merged = tf.summary.merge_all()
        summary = sess.run(merged)
        summary_writer.add_summary(summary,it)
        
        fig = plot(sample_list)
        plt.savefig('dcout10/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    #训练参数
    X_mb, _ = mnist.train.next_batch(mb_size)
    Z_noise = sample_Z(mb_size, Z_dim)
    c_noise = sample_c(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss],
                              feed_dict={X: X_mb, Z: Z_noise, c: c_noise})

    _, G_loss_curr = sess.run([G_solver, G_loss],
                              feed_dict={Z: Z_noise, c: c_noise})

    sess.run([Q_solver], feed_dict={Z: Z_noise, c: c_noise})

    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Extracting ./MNIST/train-images-idx3-ubyte.gz
Extracting ./MNIST/train-labels-idx1-ubyte.gz
Extracting ./MNIST/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/t10k-labels-idx1-ubyte.gz
Iter: 0
D loss: 1.153
G_loss: 2.791

Iter: 100
D loss: 0.7057
G_loss: 3.314

Iter: 200
D loss: 0.4314
G_loss: 3.027

Iter: 300
D loss: 0.4444
G_loss: 3.794

Iter: 400
D loss: 0.1369
G_loss: 5.105

Iter: 500
D loss: 0.2614
G_loss: 5.347

Iter: 600
D loss: 0.04523
G_loss: 6.309

Iter: 700
D loss: 0.02696
G_loss: 6.623

Iter: 800
D loss: 0.2182
G_loss: 6.384

Iter: 900
D loss: 0.05342
G_loss: 7.7

Iter: 1000
D loss: 0.2223
G_loss: 8.025

Iter: 1100
D loss: 0.1906
G_loss: 6.676

Iter: 1200
D loss: 0.1154
G_loss: 9.616

Iter: 1300
D loss: 0.1102
G_loss: 7.733

Iter: 1400
D loss: 0.1025
G_loss: 6.609

Iter: 1500
D loss: 0.04378
G_loss: 6.038

Iter: 1600
D loss: 0.05777
G_loss: 8.867

Iter: 1700
D loss: 0.1017
G_loss: 6.319

Iter: 1800
D loss: 0.03509
G_loss: 6.304

Iter: 1900
D loss: 0.01476
G_loss: 7.916

Iter: 

Iter: 19400
D loss: 0.4765
G_loss: 2.766

Iter: 19500
D loss: 0.892
G_loss: 2.565

Iter: 19600
D loss: 0.756
G_loss: 2.567

Iter: 19700
D loss: 0.7781
G_loss: 2.351

Iter: 19800
D loss: 0.526
G_loss: 1.931

Iter: 19900
D loss: 1.036
G_loss: 2.392

Iter: 20000
D loss: 0.7319
G_loss: 2.192

Iter: 20100
D loss: 0.4371
G_loss: 2.475

Iter: 20200
D loss: 0.7527
G_loss: 1.942

Iter: 20300
D loss: 0.6982
G_loss: 2.17

Iter: 20400
D loss: 1.075
G_loss: 2.02

Iter: 20500
D loss: 0.508
G_loss: 2.761

Iter: 20600
D loss: 0.7832
G_loss: 2.094

Iter: 20700
D loss: 1.034
G_loss: 2.087

Iter: 20800
D loss: 0.7643
G_loss: 2.171

Iter: 20900
D loss: 0.5786
G_loss: 2.583

Iter: 21000
D loss: 0.9366
G_loss: 2.398

Iter: 21100
D loss: 0.8833
G_loss: 1.466

Iter: 21200
D loss: 0.9588
G_loss: 2.544

Iter: 21300
D loss: 0.9277
G_loss: 2.289

Iter: 21400
D loss: 0.592
G_loss: 2.092

Iter: 21500
D loss: 0.7331
G_loss: 2.274

Iter: 21600
D loss: 0.8738
G_loss: 1.923

Iter: 21700
D loss: 0.6216
G_loss: 2.39

Ite

Iter: 39200
D loss: 0.6556
G_loss: 1.6

Iter: 39300
D loss: 1.061
G_loss: 1.12

Iter: 39400
D loss: 1.099
G_loss: 1.395

Iter: 39500
D loss: 1.212
G_loss: 1.301

Iter: 39600
D loss: 1.008
G_loss: 1.219

Iter: 39700
D loss: 1.149
G_loss: 1.261

Iter: 39800
D loss: 1.062
G_loss: 1.336

Iter: 39900
D loss: 1.047
G_loss: 1.313

Iter: 40000
D loss: 0.9596
G_loss: 1.366

Iter: 40100
D loss: 0.8721
G_loss: 1.214

Iter: 40200
D loss: 0.9762
G_loss: 1.217

Iter: 40300
D loss: 0.8364
G_loss: 1.272

Iter: 40400
D loss: 0.8714
G_loss: 1.337

Iter: 40500
D loss: 0.7499
G_loss: 1.686

Iter: 40600
D loss: 1.031
G_loss: 1.112

Iter: 40700
D loss: 1.022
G_loss: 1.42

Iter: 40800
D loss: 1.015
G_loss: 1.371

Iter: 40900
D loss: 0.7914
G_loss: 1.362

Iter: 41000
D loss: 1.329
G_loss: 1.281

Iter: 41100
D loss: 0.9498
G_loss: 1.396

Iter: 41200
D loss: 1.099
G_loss: 1.373

Iter: 41300
D loss: 1.056
G_loss: 1.296

Iter: 41400
D loss: 0.8297
G_loss: 1.63

Iter: 41500
D loss: 0.8336
G_loss: 1.454

Iter: 4160

Iter: 59100
D loss: 1.417
G_loss: 1.292

Iter: 59200
D loss: 1.048
G_loss: 1.291

Iter: 59300
D loss: 0.811
G_loss: 1.453

Iter: 59400
D loss: 0.8283
G_loss: 1.505

Iter: 59500
D loss: 1.225
G_loss: 1.063

Iter: 59600
D loss: 0.962
G_loss: 1.298

Iter: 59700
D loss: 1.238
G_loss: 1.512

Iter: 59800
D loss: 1.082
G_loss: 1.381

Iter: 59900
D loss: 1.097
G_loss: 1.41

Iter: 60000
D loss: 0.9562
G_loss: 1.17

Iter: 60100
D loss: 1.14
G_loss: 1.165

Iter: 60200
D loss: 0.9642
G_loss: 1.25

Iter: 60300
D loss: 0.805
G_loss: 1.237

Iter: 60400
D loss: 0.8873
G_loss: 1.812

Iter: 60500
D loss: 1.086
G_loss: 1.455

Iter: 60600
D loss: 1.004
G_loss: 1.555

Iter: 60700
D loss: 1.036
G_loss: 1.169

Iter: 60800
D loss: 1.209
G_loss: 1.26

Iter: 60900
D loss: 1.176
G_loss: 1.177

Iter: 61000
D loss: 1.076
G_loss: 1.218

Iter: 61100
D loss: 1.125
G_loss: 1.229

Iter: 61200
D loss: 1.162
G_loss: 1.186

Iter: 61300
D loss: 1.116
G_loss: 1.351

Iter: 61400
D loss: 1.097
G_loss: 1.321

Iter: 61500
D los

Iter: 79100
D loss: 1.174
G_loss: 1.124

Iter: 79200
D loss: 1.229
G_loss: 1.096

Iter: 79300
D loss: 0.9715
G_loss: 1.494

Iter: 79400
D loss: 1.019
G_loss: 1.388

Iter: 79500
D loss: 1.21
G_loss: 1.294

Iter: 79600
D loss: 0.9279
G_loss: 1.469

Iter: 79700
D loss: 1.274
G_loss: 1.188

Iter: 79800
D loss: 1.139
G_loss: 1.179

Iter: 79900
D loss: 1.232
G_loss: 1.397

Iter: 80000
D loss: 0.9647
G_loss: 1.116

Iter: 80100
D loss: 1.045
G_loss: 1.184

Iter: 80200
D loss: 1.398
G_loss: 1.086

Iter: 80300
D loss: 0.9931
G_loss: 1.388

Iter: 80400
D loss: 1.125
G_loss: 1.053

Iter: 80500
D loss: 1.187
G_loss: 0.9118

Iter: 80600
D loss: 0.9377
G_loss: 1.087

Iter: 80700
D loss: 1.004
G_loss: 1.41

Iter: 80800
D loss: 1.209
G_loss: 1.042

Iter: 80900
D loss: 1.027
G_loss: 1.183

Iter: 81000
D loss: 1.133
G_loss: 1.171

Iter: 81100
D loss: 0.8322
G_loss: 1.137

Iter: 81200
D loss: 1.138
G_loss: 0.9963

Iter: 81300
D loss: 1.152
G_loss: 1.056

Iter: 81400
D loss: 1.13
G_loss: 1.179

Iter: 81500

...

Iter: 603600
D loss: 0.9481
G_loss: 1.555

Iter: 603700
D loss: 0.8597
G_loss: 1.897

Iter: 603800
D loss: 0.9714
G_loss: 1.512

Iter: 603900
D loss: 0.8441
G_loss: 1.489

Iter: 604000
D loss: 0.8311
G_loss: 1.506

Iter: 604100
D loss: 0.8737
G_loss: 1.343

Iter: 604200
D loss: 0.87
G_loss: 1.593

Iter: 604300
D loss: 1.154
G_loss: 1.442

Iter: 604400
D loss: 1.008
G_loss: 1.553

Iter: 604500
D loss: 0.7455
G_loss: 1.805

Iter: 604600
D loss: 1.024
G_loss: 1.268

Iter: 604700
D loss: 0.8633
G_loss: 1.513

Iter: 604800
D loss: 0.9327
G_loss: 1.488

Iter: 604900
D loss: 1.101
G_loss: 1.444

Iter: 605000
D loss: 0.8769
G_loss: 1.407

Iter: 605100
D loss: 1.107
G_loss: 1.577

Iter: 605200
D loss: 1.036
G_loss: 1.34

Iter: 605300
D loss: 0.9219
G_loss: 1.605

Iter: 605400
D loss: 1.074
G_loss: 1.29

Iter: 605500
D loss: 0.8904
G_loss: 1.394

Iter: 605600
D loss: 0.9288
G_loss: 1.382

Iter: 605700
D loss: 0.8994
G_loss: 1.395

Iter: 605800
D loss: 1.005
G_loss: 1.273

Iter: 605900
D loss: 1.

Iter: 623000
D loss: 1.101
G_loss: 1.021

Iter: 623100
D loss: 0.9218
G_loss: 1.463

Iter: 623200
D loss: 1.07
G_loss: 1.19

Iter: 623300
D loss: 0.9278
G_loss: 1.485

Iter: 623400
D loss: 0.6926
G_loss: 1.476

Iter: 623500
D loss: 1.102
G_loss: 1.29

Iter: 623600
D loss: 0.8684
G_loss: 1.302

Iter: 623700
D loss: 1.065
G_loss: 1.347

Iter: 623800
D loss: 0.91
G_loss: 1.384

Iter: 623900
D loss: 1.078
G_loss: 1.224

Iter: 624000
D loss: 0.9636
G_loss: 1.58

Iter: 624100
D loss: 0.9414
G_loss: 1.358

Iter: 624200
D loss: 0.9322
G_loss: 1.299

Iter: 624300
D loss: 0.9382
G_loss: 1.37

Iter: 624400
D loss: 0.961
G_loss: 1.296

Iter: 624500
D loss: 0.7944
G_loss: 1.433

Iter: 624600
D loss: 0.8607
G_loss: 1.439

Iter: 624700
D loss: 0.9604
G_loss: 1.425

Iter: 624800
D loss: 1.128
G_loss: 1.43

Iter: 624900
D loss: 0.9149
G_loss: 1.564

Iter: 625000
D loss: 0.8037
G_loss: 1.286

Iter: 625100
D loss: 1.047
G_loss: 1.414

Iter: 625200
D loss: 0.9704
G_loss: 1.383

Iter: 625300
D loss: 0.6841

Iter: 642300
D loss: 0.9019
G_loss: 1.247

Iter: 642400
D loss: 0.9242
G_loss: 1.329

Iter: 642500
D loss: 0.9264
G_loss: 1.593

Iter: 642600
D loss: 0.9694
G_loss: 1.154

Iter: 642700
D loss: 0.7689
G_loss: 1.499

Iter: 642800
D loss: 1.077
G_loss: 1.284

Iter: 642900
D loss: 0.9836
G_loss: 1.494

Iter: 643000
D loss: 0.8137
G_loss: 1.618

Iter: 643100
D loss: 0.8749
G_loss: 1.223

Iter: 643200
D loss: 0.7081
G_loss: 1.492

Iter: 643300
D loss: 1.181
G_loss: 1.123

Iter: 643400
D loss: 0.9613
G_loss: 1.533

Iter: 643500
D loss: 1.175
G_loss: 1.562

Iter: 643600
D loss: 0.9367
G_loss: 1.78

Iter: 643700
D loss: 1.086
G_loss: 1.397

Iter: 643800
D loss: 0.7861
G_loss: 1.8

Iter: 643900
D loss: 0.8141
G_loss: 1.546

Iter: 644000
D loss: 0.9536
G_loss: 1.477

Iter: 644100
D loss: 0.8655
G_loss: 1.301

Iter: 644200
D loss: 0.7841
G_loss: 1.564

Iter: 644300
D loss: 0.9581
G_loss: 1.43

Iter: 644400
D loss: 0.758
G_loss: 1.647

Iter: 644500
D loss: 0.916
G_loss: 1.274

Iter: 644600
D loss: 

Iter: 661600
D loss: 0.8349
G_loss: 1.37

Iter: 661700
D loss: 1.012
G_loss: 1.381

Iter: 661800
D loss: 0.9972
G_loss: 1.6

Iter: 661900
D loss: 0.8791
G_loss: 1.266

Iter: 662000
D loss: 1.185
G_loss: 1.239

Iter: 662100
D loss: 0.9222
G_loss: 1.293

Iter: 662200
D loss: 0.9822
G_loss: 1.481

Iter: 662300
D loss: 1.011
G_loss: 1.355

Iter: 662400
D loss: 1.315
G_loss: 1.223

Iter: 662500
D loss: 1.05
G_loss: 1.412

Iter: 662600
D loss: 0.8203
G_loss: 1.467

Iter: 662700
D loss: 0.8317
G_loss: 1.496

Iter: 662800
D loss: 0.9013
G_loss: 1.474

Iter: 662900
D loss: 1.184
G_loss: 1.141

Iter: 663000
D loss: 0.9598
G_loss: 1.483

Iter: 663100
D loss: 0.7708
G_loss: 1.482

Iter: 663200
D loss: 0.7029
G_loss: 1.534

Iter: 663300
D loss: 0.9317
G_loss: 1.561

Iter: 663400
D loss: 1.005
G_loss: 1.393

Iter: 663500
D loss: 0.8601
G_loss: 1.399

Iter: 663600
D loss: 0.8689
G_loss: 1.469

Iter: 663700
D loss: 0.9619
G_loss: 1.313

Iter: 663800
D loss: 0.8672
G_loss: 1.261

Iter: 663900
D loss: 0

Iter: 680900
D loss: 0.9058
G_loss: 1.627

Iter: 681000
D loss: 0.8088
G_loss: 1.309

Iter: 681100
D loss: 1.027
G_loss: 1.368

Iter: 681200
D loss: 0.7896
G_loss: 1.41

Iter: 681300
D loss: 0.9575
G_loss: 1.591

Iter: 681400
D loss: 0.8566
G_loss: 1.446

Iter: 681500
D loss: 0.9922
G_loss: 1.437

Iter: 681600
D loss: 0.7517
G_loss: 1.477

Iter: 681700
D loss: 0.6767
G_loss: 1.284

Iter: 681800
D loss: 1.021
G_loss: 1.147

Iter: 681900
D loss: 0.7278
G_loss: 1.552

Iter: 682000
D loss: 0.7155
G_loss: 1.777

Iter: 682100
D loss: 0.8915
G_loss: 1.535

Iter: 682200
D loss: 0.8042
G_loss: 1.603

Iter: 682300
D loss: 0.9095
G_loss: 1.241

Iter: 682400
D loss: 1.122
G_loss: 1.362

Iter: 682500
D loss: 0.7365
G_loss: 1.604

Iter: 682600
D loss: 0.8789
G_loss: 1.604

Iter: 682700
D loss: 0.6684
G_loss: 1.637

Iter: 682800
D loss: 1.138
G_loss: 1.019

Iter: 682900
D loss: 0.8403
G_loss: 1.661

Iter: 683000
D loss: 0.979
G_loss: 1.205

Iter: 683100
D loss: 0.9396
G_loss: 1.303

Iter: 683200
D lo

In [ ]:
D_fake

In [ ]:
Z

In [ ]:
G_sample

In [ ]:
np.random.randint(0, 10)

In [ ]:
idx = np.random.randint(0, 10) #0-9的整数
c_noise = np.zeros([16, 10])
c_noise[range(16), idx] = 1
c_noise[range(16), idx+1] = 1
c_noise

In [ ]:
list(enumerate(c_noise))

In [ ]:
c_noise[:,0]

In [ ]:
import numpy as np
np.random.randint(0, 10)

投掷骰子，在12个位置中扔1次，每个位置中扔到骰子的次数。这里起到位置随机取1的作用。

In [ ]:
np.random.multinomial(1, 12*[1/12], size=mb_size)

In [ ]:
movie=[1,2,3]
movie[1]

In [ ]:
movie=[]
for i in range(12):
    movie.append(i)
movie